# ILP solving the 1D cutting stock problem 

Here I am minimizing the number of bars used and the waste loss, per each bar.

In [10]:
import pulp as pl
import math
import time

In [12]:
def solve_cutting_stock(
    L,                       # stock length
    item_lengths,            # list  l_i
    demands,                 # list  d_i
    cost_bar=1.0,            # c_bar
    cost_waste=1e-4,         # c_w - usually small 
    bigM=None,              
    solver=None,
    report_filename=None
):
    n_items = len(item_lengths)
    assert n_items == len(demands)

    if bigM is None:
        # LB = total required length / stock length
        LB = math.ceil(sum(d * l for l, d in zip(demands, item_lengths)) / L)
        # small safety buffer, e.g. 5 %
        bigM = math.ceil(1.05 * LB)

    bars = range(bigM)
    items = range(n_items)

    prob = pl.LpProblem("CuttingStock_BarIndexed", pl.LpMinimize)

    u = pl.LpVariable.dicts("useBar", bars, lowBound=0, upBound=1, cat="Binary")
    y = pl.LpVariable.dicts(
        "cut", [(i, j) for i in items for j in bars],
        lowBound=0, cat="Integer"
    )
    s = pl.LpVariable.dicts("trim", bars, lowBound=0, cat="Continuous")

    # OBJECTIVE
    prob += pl.lpSum(cost_bar * u[j] + cost_waste * s[j] for j in bars), "TotalCost"

    # DEMAND
    for i in items:
        prob += pl.lpSum(y[i, j] for j in bars) >= demands[i], f"demand_{i}"

    # Capacity on each bar: Σ l_i * y_ij + s_j = L * u_j
    for j in bars:
        prob += (
            pl.lpSum(item_lengths[i] * y[i, j] for i in items) + s[j]
            == L * u[j],
            f"capacity_{j}",
        )
    for j in range(bigM-1):                                      
        prob += (L * u[j] - s[j]) >= (L * u[j+1] - s[j+1]), f"sym_{j}"

    if solver is None:
        solver = pl.PULP_CBC_CMD(msg=False, presolve='on', timeLimit=300)
    prob.solve(solver)
    #print(f"Status: {pl.LpStatus[prob.status]}")
    #print(f"Objective value = {pl.value(prob.objective):.2f}")

    # REPORT
    output_lines = [] # start with a list and append everything
    output_lines.append(f"Status: {pl.LpStatus[prob.status]}")
    output_lines.append(f"Objective value = {pl.value(prob.objective):.2f}")
    used_bars = [j for j in bars if pl.value(u[j]) > 0.5]
    output_lines.append(f"Bars used = {len(used_bars)} of {bigM}\n")

    for j in used_bars:
        trim = pl.value(s[j])
        cuts = [(i, int(pl.value(y[i, j]))) for i in items if pl.value(y[i, j]) > 0.5]
        pattern_str = ", ".join(f"{qty}×{item_lengths[i]}" for i, qty in cuts)
        output_lines.append(f"Bar {j:>2}: {pattern_str:<25}  trim = {trim:.2f}")

    report = "\n".join(output_lines) # join the list together in the report
    print(report)

    if report_filename is not None:
        with open(report_filename, "w") as f:
            f.write(report)

    return prob

In [14]:
%%time

stock_length = 6000                      # mm
lengths     = [1820, 2350, 410]          # mm
demands     = [50,   35,   40]          # required counts of each

solve_cutting_stock(
    L=stock_length,
    item_lengths=lengths,
    demands=demands,
    cost_bar=1.0,
    cost_waste=1e-4, 
    report_filename="stock_report.txt"
)

Status: Optimal
Objective value = 32.11
Bars used = 32 of 34

Bar  0: 2×1820, 1×2350             trim = 10.00
Bar  1: 2×1820, 1×2350             trim = 10.00
Bar  2: 2×1820, 1×2350             trim = 10.00
Bar  3: 2×1820, 1×2350             trim = 10.00
Bar  4: 2×1820, 1×2350             trim = 10.00
Bar  5: 2×1820, 1×2350             trim = 10.00
Bar  6: 2×1820, 1×2350             trim = 10.00
Bar  7: 2×1820, 1×2350             trim = 10.00
Bar  8: 2×1820, 1×2350             trim = 10.00
Bar  9: 2×1820, 1×2350             trim = 10.00
Bar 10: 2×1820, 1×2350             trim = 10.00
Bar 11: 2×1820, 1×2350             trim = 10.00
Bar 12: 2×1820, 1×2350             trim = 10.00
Bar 13: 2×1820, 1×2350             trim = 10.00
Bar 14: 2×1820, 1×2350             trim = 10.00
Bar 15: 2×1820, 1×2350             trim = 10.00
Bar 16: 2×1820, 1×2350             trim = 10.00
Bar 17: 2×1820, 1×2350             trim = 10.00
Bar 18: 2×1820, 1×2350             trim = 10.00
Bar 19: 2×1820, 1×2350    

CuttingStock_BarIndexed:
MINIMIZE
0.0001*trim_0 + 0.0001*trim_1 + 0.0001*trim_10 + 0.0001*trim_11 + 0.0001*trim_12 + 0.0001*trim_13 + 0.0001*trim_14 + 0.0001*trim_15 + 0.0001*trim_16 + 0.0001*trim_17 + 0.0001*trim_18 + 0.0001*trim_19 + 0.0001*trim_2 + 0.0001*trim_20 + 0.0001*trim_21 + 0.0001*trim_22 + 0.0001*trim_23 + 0.0001*trim_24 + 0.0001*trim_25 + 0.0001*trim_26 + 0.0001*trim_27 + 0.0001*trim_28 + 0.0001*trim_29 + 0.0001*trim_3 + 0.0001*trim_30 + 0.0001*trim_31 + 0.0001*trim_32 + 0.0001*trim_33 + 0.0001*trim_4 + 0.0001*trim_5 + 0.0001*trim_6 + 0.0001*trim_7 + 0.0001*trim_8 + 0.0001*trim_9 + 1.0*useBar_0 + 1.0*useBar_1 + 1.0*useBar_10 + 1.0*useBar_11 + 1.0*useBar_12 + 1.0*useBar_13 + 1.0*useBar_14 + 1.0*useBar_15 + 1.0*useBar_16 + 1.0*useBar_17 + 1.0*useBar_18 + 1.0*useBar_19 + 1.0*useBar_2 + 1.0*useBar_20 + 1.0*useBar_21 + 1.0*useBar_22 + 1.0*useBar_23 + 1.0*useBar_24 + 1.0*useBar_25 + 1.0*useBar_26 + 1.0*useBar_27 + 1.0*useBar_28 + 1.0*useBar_29 + 1.0*useBar_3 + 1.0*useBar_30 + 1.